# Semantic Kernel Basics - Getting Started with AI Agents

Welcome to the world of Semantic Kernel! This notebook will guide you through the fundamentals of building AI agents using Microsoft's Semantic Kernel SDK.

## What is Semantic Kernel?

Semantic Kernel is a lightweight, open-source development kit that lets you easily build AI agents and integrate the latest AI models into your Python, C#, or Java codebase. It serves as an efficient middleware that enables rapid delivery of enterprise-grade AI solutions.

### Key Features:
- **Model Flexibility**: Built-in support for OpenAI, Azure OpenAI, Hugging Face, NVIDIA, and more
- **Agent Framework**: Build modular AI agents with access to tools/plugins, memory, and planning capabilities
- **Plugin Ecosystem**: Support for native code functions, prompt templates, OpenAPI specs, or Model Context Protocol (MCP)
- **Production Ready**: Version 1.0+ support with commitment to non-breaking changes

## Resources
- [Official Documentation](https://learn.microsoft.com/en-us/semantic-kernel/)
- [GitHub Repository](https://github.com/microsoft/semantic-kernel)
- [PyPI Package](https://pypi.org/project/semantic-kernel/)

Let's get started!

## Installation and Setup

First, let's install the required packages and set up our environment.

In [ ]:
# Install required packages
# !pip install semantic-kernel python-dotenv

## Environment Configuration

You'll need to configure your AI service. Create a `.env` file in your project root with the following configurations:

### Azure OpenAI models

```
# Azure OpenAI Configuration - Default Model (GPT-4.1)
AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4.1
AZURE_OPENAI_ENDPOINT=<>
AZURE_OPENAI_API_KEY=<>

# Azure OpenAI Configuration - Reasoning Model (O4-mini)
AZURE_REASONING_DEPLOYMENT_NAME=o4-mini
AZURE_REASONING_ENDPOINT=<>
AZURE_REASONING_API_KEY=<>
```

For this tutorial, we'll use Azure OpenAI, but you can easily switch to OpenAI by uncommenting the relevant lines.

In [ ]:
import asyncio
try:
    from typing import Annotated
except ImportError:
    from typing_extensions import Annotated

import os
from dotenv import load_dotenv

# Semantic Kernel imports
from semantic_kernel import Kernel
from semantic_kernel.agents import Agent, ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings, AzureChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions import kernel_function

# Load environment variables from .env file
load_dotenv()

print("✅ Environment setup complete!")

✅ Environment setup complete!


## 1. Setting Up Chat Completion

The foundation of Semantic Kernel is the chat completion service. This is how we connect to the AI models.

In [4]:
# Option 1: Azure OpenAI (recommended for enterprise)
chat_completion = AzureChatCompletion(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
)

# Option 2: OpenAI (uncomment to use)
# chat_completion = OpenAIChatCompletion(
#     api_key=os.getenv("OPENAI_API_KEY"),
#     ai_model_id=os.getenv("OPENAI_MODEL_ID", "gpt-4o-mini")
# )

# Other supported providers include: Azure AI inference, Bedrock, Anthropic, Vertex, Mistral, Ollama, and more.

print("✅ Chat completion service configured!")

✅ Chat completion service configured!


## 2. Basic Chat Interaction

Let's start with the simplest possible interaction - sending a message and getting a response.

In [5]:
# Create execution settings (optional - for controlling temperature, max tokens, etc.)
execution_settings = AzureChatPromptExecutionSettings(
    max_completion_tokens=2000
)

# Create a simple chat history
chat_history = ChatHistory()
chat_history.add_user_message("Hello, how are you?")

# Get a response from the AI
response = await chat_completion.get_chat_message_content(
    chat_history=chat_history, 
    settings=execution_settings
)

print(f"AI Response: {response.content}")

AI Response: Hello! I’m doing well, thank you. How can I help you today?


## 3. Working with System Messages and Roles

System messages help you set the behavior and personality of your AI assistant. Let's create a more sophisticated interaction.

In [6]:
# Create a new chat history with a system message
chat_history = ChatHistory()

# Add a system message to set the AI's behavior
chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.SYSTEM,
        content="You are a helpful assistant that always answers in a friendly and encouraging tone. Keep responses concise but warm."
    )
)

# Add a user message
chat_history.add_user_message("I'm learning about AI development. Any tips?")

# Get the response
response = await chat_completion.get_chat_message_content(
    chat_history=chat_history, 
    settings=execution_settings
)

print(f"AI Response: {response.content}")

AI Response: Here are a few tips to guide your AI-development journey:

• Build a strong foundation  
  – Learn Python and brush up on linear algebra, probability and basic calculus.  

• Understand core ML concepts  
  – Study supervised vs. unsupervised learning, overfitting/underfitting and model evaluation metrics.  

• Get hands-on with frameworks  
  – Experiment in small projects using PyTorch or TensorFlow; follow official tutorials.  

• Explore datasets and challenges  
  – Try Kaggle competitions or public datasets to practice data cleaning, feature engineering and model tuning.  

• Read and stay curious  
  – Follow top AI blogs, research papers (arXiv) and newsletters to keep up with new ideas.  

• Join a community  
  – Participate in forums like Stack Overflow, Reddit’s r/MachineLearning or local meetups to ask questions and share knowledge.  

• Keep ethics in mind  
  – Think about bias, fairness and real-world impact as you design and deploy your models.  

Every sm

## 4. Multimodal Interactions (Text + Images)

Modern AI models can process both text and images. Let's try sending an image along with our message.

**Note**: Make sure you have an image file in your project directory, or update the path below.

See the image we are using:

<details>

<img src="resources/france.png" width="800" />

</details>

In [ ]:
# Create a multimodal message with both text and image
chat_history = ChatHistory()

# Add system message for image analysis
chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.SYSTEM,
        content="You are an expert at analyzing images. Provide detailed but concise descriptions."
    )
)

# Add a multimodal user message
try:
    chat_history.add_message(
        ChatMessageContent(
            role=AuthorRole.USER,
            name="User",
            items=[
                TextContent(text="What can you tell me about this image?"),
                ImageContent.from_file(file_path="resources/france.png", mime_type="image/png")
            ]
        )
    )
    
    response = await chat_completion.get_chat_message_content(
        chat_history=chat_history, 
        settings=execution_settings
    )
    
    print(f"AI Response: {response.content}")
    
except FileNotFoundError:
    print("⚠️  Image file 'france.png' not found. Skipping multimodal example.")
    print("💡 To test this feature, add an image file to your project directory and update the path.")

AI Response: The image shows a clear, sunny day in Paris with the Eiffel Tower rising prominently in the background. In the foreground, a French tricolore flag (blue, white, red vertical stripes) is mounted on a slender white pole, fluttering gently in the breeze. Below and around the tower, neatly trimmed green lawns and tree-lined walkways are dotted with small groups of people—tourists and locals—strolling, taking photos or relaxing. The sky is a bright blue with a few wispy white clouds, lending a vibrant contrast to the iron lattice structure of the monument.


## 5. Introduction to Agents

Agents are the core building blocks of Semantic Kernel. They encapsulate AI models with specific instructions and capabilities.

In [7]:
# Create your first agent
agent = ChatCompletionAgent(
    service=chat_completion,
    name="AssistantBot",
    instructions="You are a knowledgeable assistant that provides helpful answers in one or two sentences. Be concise but informative."
)

# Test the agent
response = await agent.get_response(messages="What is the capital of France?")
print(f"Agent Response: {response.message.content}")

Agent Response: The capital of France is Paris.


## 6. Working with Conversation Threads

Threads allow you to maintain conversation context across multiple interactions. This is essential for building conversational experiences.

In [ ]:
# Create an agent for conversation
conversational_agent = ChatCompletionAgent(
    service=chat_completion,
    name="ConversationBot",
    instructions="You are a friendly conversational assistant."
)

# Start a conversation without a thread
thread: ChatHistoryAgentThread = None
response = await conversational_agent.get_response(
    messages="Hi, I'm planning a trip to Japan. Any recommendations?", 
    thread=thread
)

print(f"Bot: {response.message.content}")
print("-" * 50)

Bot: Here are some ideas to get you started—feel free to let me know more about your travel dates, trip length or interests, and I can tailor these suggestions further:

1. Major Destinations  
  • Tokyo: ultra-modern city meets traditional temples (Senso-ji in Asakusa, Meiji Shrine in Harajuku), plus neighborhoods like Shibuya, Shinjuku, Akihabara and Odaiba.  
  • Kyoto: over 1,000 temples and shrines, iconic Fushimi Inari Taisha gates, traditional tea houses in Gion and the Golden Pavilion (Kinkaku-ji).  
  • Osaka: known for its street food (takoyaki, okonomiyaki), Dōtonbori nightlife and Osaka Castle.  

2. Off-the-Beaten-Path & Regional Gems  
  • Hiroshima & Miyajima: Peace Memorial Park and the “floating” Itsukushima Shrine.  
  • Kanazawa: beautifully preserved samurai and geisha districts, Kenrokuen Garden.  
  • Takayama & Shirakawa-go: traditional mountain villages and thatched-roof farmhouses.  
  • Hokkaido (if you travel in summer, it’s cooler; winter offers world-class 

In [9]:
# Continue the conversation using the thread
thread = response.thread  # Get the thread from the previous response

response = await conversational_agent.get_response(
    messages="What about food recommendations?", 
    thread=thread
)

print(f"Bot: {response.message.content}")
print("-" * 50)

Bot: Here’s a tasty rundown of must-try eats—and where to find them—on your trip around Japan. Feel free to mix and match by region or mood!

1. Sushi & Sashimi  
 • Tsukiji/Toyosu (Tokyo) – try an early-morning sushi counter; go for an “omakase” set if you can.  
 • Conveyor-belt sushi (kaiten-zushi) – cheap and fun; Mid-tier chains like Sushiro or Kura are everywhere.  

2. Ramen (by region)  
 • Tokyo (shoyu ramen) – famous shops like Ichiran, Afuri (yuzu-flavored broth), or local mom-and-pop shops.  
 • Sapporo (miso ramen) – hearty, topped with corn & butter; check out Ramen Shingen or Sumire.  
 • Hakata (Fukuoka) tonkotsu – rich pork broth; Ichiran’s original shop or Ippudo.  
 • Kitakata (Fukushima) – light shoyu broth & thick noodles.  

3. Osaka Street Food  
 • Takoyaki (octopus balls) – head to Dōtonbori or street stalls all over Namba.  
 • Okonomiyaki – Osaka style (mixed pancake); try Mizuno or Kiji in the Umeda area.  

4. Hiroshima-Style Okonomiyaki  
 • Layered (noodl

In [10]:
# Test context awareness
response = await conversational_agent.get_response(
    messages="What was the topic of our conversation?", 
    thread=thread
)

print(f"Bot: {response.message.content}")
print("-" * 50)

Bot: We’ve been talking about planning your trip to Japan—covering top destinations and itineraries first, then diving into local food must-tries and where to find them.
--------------------------------------------------


## 7. Exploring Thread Messages

You can inspect the conversation history stored in a thread.

In [11]:
# View all messages in the thread
print("🔍 Conversation History:")
print("=" * 50)

async for message in thread.get_messages():
    role = message.role.value.upper()
    name = f" ({message.name})" if message.name else ""
    print(f"[{role}{name}]: {message.content}")
    print("-" * 30)

🔍 Conversation History:
[USER]: Hi, I'm planning a trip to Japan. Any recommendations?
------------------------------
[ASSISTANT (ConversationBot)]: Here are some ideas to get you started—feel free to let me know more about your travel dates, trip length or interests, and I can tailor these suggestions further:

1. Major Destinations  
  • Tokyo: ultra-modern city meets traditional temples (Senso-ji in Asakusa, Meiji Shrine in Harajuku), plus neighborhoods like Shibuya, Shinjuku, Akihabara and Odaiba.  
  • Kyoto: over 1,000 temples and shrines, iconic Fushimi Inari Taisha gates, traditional tea houses in Gion and the Golden Pavilion (Kinkaku-ji).  
  • Osaka: known for its street food (takoyaki, okonomiyaki), Dōtonbori nightlife and Osaka Castle.  

2. Off-the-Beaten-Path & Regional Gems  
  • Hiroshima & Miyajima: Peace Memorial Park and the “floating” Itsukushima Shrine.  
  • Kanazawa: beautifully preserved samurai and geisha districts, Kenrokuen Garden.  
  • Takayama & Shirakawa-

## 8. Creating Your First Plugin

Plugins extend agents with custom functionality. Let's create a simple menu plugin for a restaurant.

In [12]:
class RestaurantMenuPlugin:
    """A plugin that provides restaurant menu information."""

    @kernel_function(description="Get today's special menu items.")
    def get_specials(self) -> Annotated[str, "Returns today's special menu items."]:
        return """
        🍲 Special Soup: Clam Chowder
        🥗 Special Salad: Caesar Salad with Grilled Chicken
        🍹 Special Drink: Fresh Mint Lemonade
        🍰 Special Dessert: Chocolate Lava Cake
        """

    @kernel_function(description="Get the price of a specific menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        # In a real application, this would query a database
        prices = {
            "clam chowder": "$12.99",
            "caesar salad": "$14.99",
            "mint lemonade": "$4.99",
            "chocolate lava cake": "$8.99"
        }
        
        item_key = menu_item.lower()
        return prices.get(item_key, "$9.99")  # Default price

    @kernel_function(description="Check if a menu item is available.")
    def check_availability(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns availability status."]:
        # Simulate checking availability
        available_items = ["clam chowder", "caesar salad", "mint lemonade", "chocolate lava cake"]
        item_key = menu_item.lower()
        
        if item_key in available_items:
            return f"✅ {menu_item} is available!"
        else:
            return f"❌ {menu_item} is not available today."

print("✅ Restaurant Menu Plugin created!")

✅ Restaurant Menu Plugin created!


## 9. Using Plugins with Agents

Now let's create an agent that uses our restaurant menu plugin.

In [13]:
# Create an agent with the menu plugin
restaurant_agent = ChatCompletionAgent(
    service=chat_completion,
    name="RestaurantHost",
    instructions="You are a friendly restaurant host. Help customers with menu questions, prices, and availability. Be welcoming and helpful!",
    plugins=[RestaurantMenuPlugin()]
)

print("✅ Restaurant agent with menu plugin created!")

✅ Restaurant agent with menu plugin created!


## 10. Testing the Plugin-Enabled Agent

Let's have a conversation with our restaurant agent to see the plugin in action.

In [14]:
# Test the agent with plugin functionality
customer_requests = [
    "Hello! I'd like to know about today's specials.",
    "How much does the clam chowder cost?",
    "Is the chocolate lava cake available?",
    "What about pizza? Is that available?"
]

thread = None

for request in customer_requests:
    print(f"👤 Customer: {request}")
    
    response = await restaurant_agent.get_response(messages=request, thread=thread)
    thread = response.thread
    
    print(f"🍽️ Host: {response.message.content}")
    print("-" * 60)

👤 Customer: Hello! I'd like to know about today's specials.
🍽️ Host: Hello there! Here are today’s specials:

- Special Soup: Clam Chowder  
- Special Salad: Caesar Salad with Grilled Chicken  
- Special Drink: Fresh Mint Lemonade  
- Special Dessert: Chocolate Lava Cake  

Feel free to ask about any of these or anything else on the menu. Enjoy!
------------------------------------------------------------
👤 Customer: How much does the clam chowder cost?
🍽️ Host: The Clam Chowder costs $12.99. Would you like to order it, or is there anything else I can help you with?
------------------------------------------------------------
👤 Customer: Is the chocolate lava cake available?
🍽️ Host: Good news! The Chocolate Lava Cake is available. Would you like to add it to your order or have any other questions?
------------------------------------------------------------
👤 Customer: What about pizza? Is that available?
🍽️ Host: I’m sorry, but pizza isn’t available today. Can I interest you in one o

## 11. Understanding Function Calls

Let's examine what happens behind the scenes when the agent uses our plugin functions.

In [ ]:
# Let's look at the conversation history to see function calls
print("🔍 Detailed Conversation History (including function calls):")
print("=" * 70)

async for message in thread.get_messages():
    message_dict = message.to_dict()
    
    if message_dict.get('role') == 'user':
        print(f"👤 USER: {message_dict['content']}")
    elif message_dict.get('role') == 'assistant':
        if 'tool_calls' in message_dict:
            print(f"🤖 AGENT: [Making function call]")
            for tool_call in message_dict['tool_calls']:
                function_name = tool_call['function']['name']
                arguments = tool_call['function']['arguments']
                print(f"   📞 Calling: {function_name}({arguments})")
        else:
            print(f"🤖 AGENT: {message_dict['content']}")
    elif message_dict.get('role') == 'tool':
        print(f"🔧 FUNCTION RESULT: {message_dict['content']}")
    
    print("-" * 40)

🔍 Detailed Conversation History (including function calls):
👤 USER: Hello! I'd like to know about today's specials.
----------------------------------------
🤖 AGENT: [Making function call]
   📞 Calling: RestaurantMenuPlugin-get_specials({})
----------------------------------------
🔧 FUNCTION RESULT: 
        🍲 Special Soup: Clam Chowder
        🥗 Special Salad: Caesar Salad with Grilled Chicken
        🍹 Special Drink: Fresh Mint Lemonade
        🍰 Special Dessert: Chocolate Lava Cake
        
----------------------------------------
🤖 AGENT: Hello there! Here are today’s specials:

- Special Soup: Clam Chowder  
- Special Salad: Caesar Salad with Grilled Chicken  
- Special Drink: Fresh Mint Lemonade  
- Special Dessert: Chocolate Lava Cake  

Feel free to ask about any of these or anything else on the menu. Enjoy!
----------------------------------------
👤 USER: How much does the clam chowder cost?
----------------------------------------
🤖 AGENT: [Making function call]
   📞 Calling:

## 12. Cleanup

It's good practice to clean up threads when you're done with them.

In [16]:
# Clean up the thread
if thread:
    await thread.delete()
    print("✅ Thread cleaned up successfully!")

✅ Thread cleaned up successfully!


## 🎉 Congratulations!

You've successfully completed the Semantic Kernel Basics tutorial! Here's what you've learned:

### ✅ Key Concepts Covered:
1. **Environment Setup** - Configuring Semantic Kernel with Azure OpenAI or OpenAI
2. **Chat Completion** - Basic AI interactions
3. **System Messages** - Controlling AI behavior and personality
4. **Multimodal Input** - Working with text and images
5. **Agents** - Creating AI assistants with specific instructions
6. **Conversation Threads** - Maintaining context across interactions
7. **Plugins** - Extending agents with custom functionality
8. **Function Calling** - How agents use tools to solve problems

### 🚀 Next Steps:
Ready to dive deeper? Check out the **Advanced Semantic Kernel** notebook to learn about:
- Multi-agent orchestration
- Streaming responses
- Complex workflows
- Human-in-the-loop systems

### 📚 Additional Resources:
- [Semantic Kernel Documentation](https://learn.microsoft.com/en-us/semantic-kernel/)
- [Python SDK Reference](https://learn.microsoft.com/en-us/semantic-kernel/get-started/quick-start-guide)
- [Agent Framework Guide](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/)
- [Plugin Development](https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/)

Happy coding! 🚀